# Analysis of Mumbai neighbourhoods Data and Finding Best Location for opening new coffee shop. 

## Capstone Project - The Battle of the Neighborhoods (Week 2)


##Table of contents
####Introduction: 
####Business Problem
####Data 
####Methodology
####Analysis
####Results and Discussion
####Conclusion

###Introduction:
####Business Problem:
In this project we will try to find an optimal location for a coffe shope. Specifically, this report will be targeted to stakeholders interested in opening an coffee shop  in Mumbai , India.

Since there are lots of coffee in Mumbai we will try to detect locations that are not already crowded with restaurants and coffee shop. We are also particularly interested in areas with no coffee shop in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

Importing requred lib file

In [0]:
#!pip install beautifulsoup4

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import pandas as pd
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


Getting requred dataframe  of  List of neighbourhoods in Mumbai. scraping web page

In [0]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai").text
soup = BeautifulSoup(req, 'lxml')
div = soup.find('div', class_="mw-parser-output" )
print("web Page Imported")
#Code to extract the relevent data from the request object using beautiful soup
data = pd.DataFrame(columns=['Borough','Neighborhood'])
i=-1
flag = False
no=0
prev_borough = None
for child in div.children:
    if child.name:
        span = child.find('span')
        if span!=-1 and span is not None:
            try:
                if span['class'][0] == 'mw-headline' and child.a.text!='edit':
                    prev_borough = child.a.text
                    i+=1
                    flag = True
                    continue
            except KeyError:
                continue
        if child.name=='ul' and flag==True:
            neighborhood = []
            for ch in child.children:
                
                try:
                    data.loc[no]=[prev_borough,ch.text]
                    no+=1
                except AttributeError:
                    pass
        flag = False
data[50:60]
dfn= pd.DataFrame(data)
dfn.shape

web Page Imported


(99, 2)

In [0]:
dfn.to_csv('/content/mumbai_csv.csv', index=False)

Getting Location Data From Geolocator by providing List_of_neighbourhoods_in_Mumbai data 

In [0]:
lat_lng = pd.DataFrame(columns=['latitude','longitude'])
geolocator = Nominatim(user_agent="ny_explorer")
for i in range(99):
    address = data['Neighborhood'].loc[i]+',Mumbai'
    try: 
        location = geolocator.geocode(address)
        lat_lng.loc[i]=[location.latitude,location.longitude]
    except AttributeError:
        continue
df1 = data
df2 = lat_lng
mumbai_neighbourhood_data = pd.concat([df1, df2], axis=1)

In [0]:
mumbai_neighbourhood_data.head()

,Borough,Neighborhood,latitude,longitude
0,Andheri,Amboli,19.127931,72.847735
1,Andheri,Chakala,19.115287,72.861808
2,Andheri,D.N. Nagar,19.128292,72.830193
3,Andheri,Four Bungalows,19.128794,72.825554
4,Andheri,JB Nagar,NaN,NaN


In [0]:
mumbai_neighbourhood_data.shape

(99, 4)

In [0]:
mumbai_neighbourhood_data.to_csv('/content/mumbai_csv.csv', index=False)

In [0]:
df=pd.read_csv('/content/mumbai_csv.csv')
df.shape

(99, 4)

In [0]:
df.head()

,Borough,Neighborhood,latitude,longitude
0,Andheri,Amboli,19.127931,72.847735
1,Andheri,Chakala,19.115287,72.861808
2,Andheri,D.N. Nagar,19.128292,72.830193
3,Andheri,Four Bungalows,19.128794,72.825554
4,Andheri,JB Nagar,19.111100,72.865600


## Analysis of neighbourhoods using map 
Creating Folium Map of List of neighbourhoods in Mumbai

In [0]:
latitude = df.loc[0, 'latitude']
longitude = df.loc[0, 'longitude']
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['latitude'],df['longitude'],df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

![Clusters](https://github.com/samir-cell/fantastic-datascience-Coursera_Capstone/blob/master/img/cluster.JPG?raw=true)

Getting Data Using Foursquare API by providing ID and Secret.

In [0]:
CLIENT_ID = 'MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR' # your Foursquare ID
CLIENT_SECRET = '10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MGMBVNJKXHKVKCPQNXRD401BWMBZ4YQUF3JDHG5P3FEANLHR
CLIENT_SECRET:10RLP4RHEHTAEXBIOOEP3NV1YYZQA3VMLVGDW3N1UDSQOFTA


In [0]:
latitude = df.loc[0, 'latitude']
longitude = df.loc[0, 'longitude']
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()


In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
from pandas.io.json import json_normalize

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Cafe Arfa,Indian Restaurant,19.128930,72.847140
1,Narayan Sandwich,Sandwich Place,19.121398,72.850270
2,Merwans Cake shop,Bakery,19.119300,72.845418
3,"5 Spice , Bandra",Chinese Restaurant,19.130421,72.847206
4,Shawarma Factory,Falafel Restaurant,19.124591,72.840398


In [0]:
nearby_venues.shape

(18, 4)

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
Mumbai_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Amboli
Chakala
D.N. Nagar
Four Bungalows
JB Nagar
Lokhandwala
Marol
Model Town
Oshiwara
Poonam Nagar
Sahar
Saki Naka
Seven Bungalows
Versova
Mira Road
Uttaran
Kashimira
Bandra Kurla Complex
Bandstand Promenade
Gandhi Nagar
Kherwadi
Land's End
Pali Hill
Old Town
Bandra Reclamation
Aarey Milk Colony
Motilal Nagar
Bangur Nagar
Gokuldham
Jayprakash Nagar
Pandurangwadi
NESCO colony
Jogeshwari West
Jogeshwari East
Charkop
Poisar
Hindustan Naka
Mahavir Nagar
Samta nagar
Damu nagar
Thakur complex
Thakur village
Lokandwala township
Akurli road
Janupada
Hanuman nagar
Pali Naka
Khar Danda
Dindoshi
Sunder Nagar
Pathanwadi
Rani Sati Marg
Malvani
Kalina
Vakola
Naigaon
Nalasopara
Irla
Shivaji Talav
Hanuman Nagar
Pratap Nagar
Maharastra Nagar
JAmil NAgar
Utkarsh Nagar
samarth NAgar
Sahyadri NAgar
Sarvoday NAgar
Amrut Nagar
Asalfa
Garodia Nagar
Jagdusha Nagar
Pant Nagar
Nehru Nagar
Nahur
Chandivali
Hiranandani Gardens
Indian Institute of Technology Bombay campus
Gandhi Nagar
Kannamwar Nagar
Vikhroli Pa

In [0]:
print(Mumbai_venues.shape)
Mumbai_venues.head()

(3789, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amboli,19.127931,72.847735,Cafe Arfa,19.128930,72.847140,Indian Restaurant
1,Amboli,19.127931,72.847735,Narayan Sandwich,19.121398,72.850270,Sandwich Place
2,Amboli,19.127931,72.847735,Merwans Cake shop,19.119300,72.845418,Bakery
3,Amboli,19.127931,72.847735,"5 Spice , Bandra",19.130421,72.847206,Chinese Restaurant
4,Amboli,19.127931,72.847735,Shawarma Factory,19.124591,72.840398,Falafel Restaurant


In [0]:
import pandas as pd

Mumbai_venues.to_csv('/content/mumbai_venues.csv', index=False)


In [0]:
Mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aarey Milk Colony,4,4,4,4,4,4
Akurli road,15,15,15,15,15,15
Amboli,18,18,18,18,18,18
Amrut Nagar,53,53,53,53,53,53
Asalfa,15,15,15,15,15,15
...,...,...,...,...,...,...
Uttaran,24,24,24,24,24,24
Vakola,17,17,17,17,17,17
Versova,76,76,76,76,76,76


In [0]:
print('There are {} uniques categories.'.format(len(Mumbai_venues['Venue Category'].unique())))

There are 247 uniques categories.


Creating Oneho encoding

In [0]:
Mumbai_onehot = pd.get_dummies(Mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Mumbai_onehot['Neighborhood'] = Mumbai_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Mumbai_onehot.columns[-1]] + list(Mumbai_onehot.columns[:-1])
Mumbai_onehot = Mumbai_onehot[fixed_columns]

Mumbai_onehot.head()

,Zoo,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
Mumbai_onehot.shape

(3789, 247)

In [0]:
Mumbai_grouped = Mumbai_onehot.groupby('Neighborhood').mean().reset_index()
Mumbai_grouped

,Neighborhood,Zoo,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Aarey Milk Colony,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,Akurli road,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,Amboli,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.055556,0.055556,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Amrut Nagar,0.0,0.018868,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.037736,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.018868,0.0,0.018868,0.000000,0.000000,0.018868,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.037736,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.018868,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.018868,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.037736,0.0,0.0,0.000000,0.018868,0.0
4,Asalfa,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Uttaran,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000

getting Freq of TOP 5  venues.

In [0]:
num_top_venues = 5

for hood in Mumbai_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Mumbai_grouped[Mumbai_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aarey Milk Colony----
                  venue  freq
0                  Lake  0.25
1     Indian Restaurant  0.25
2  Fast Food Restaurant  0.25
3   Monument / Landmark  0.25
4           Pastry Shop  0.00


----Akurli road----
                   venue  freq
0         Ice Cream Shop  0.20
1      Indian Restaurant  0.20
2            Coffee Shop  0.13
3                 Bakery  0.13
4  General Entertainment  0.07


----Amboli----
                venue  freq
0   Indian Restaurant  0.11
1              Bakery  0.11
2      Sandwich Place  0.11
3  Chinese Restaurant  0.06
4  Athletics & Sports  0.06


----Amrut Nagar----
               venue  freq
0               Café  0.11
1  Indian Restaurant  0.08
2             Lounge  0.06
3     Clothing Store  0.06
4        Pizza Place  0.04


----Asalfa----
                venue  freq
0                Café  0.13
1     Motorcycle Shop  0.07
2             Factory  0.07
3  Light Rail Station  0.07
4          Restaurant  0.07


----Ballard Estate----
       

### Getting top 10 venues of all neighbourhoods.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [0]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Mumbai_grouped['Neighborhood']

for ind in np.arange(Mumbai_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Mumbai_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aarey Milk Colony,Monument / Landmark,Fast Food Restaurant,Indian Restaurant,Lake,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio
1,Akurli road,Ice Cream Shop,Indian Restaurant,Coffee Shop,Bakery,Food Truck,Sandwich Place,General Entertainment,Dessert Shop,Arts & Crafts Store,Film Studio
2,Amboli,Bakery,Sandwich Place,Indian Restaurant,Camera Store,Chinese Restaurant,Falafel Restaurant,Athletics & Sports,Asian Restaurant,Coffee Shop,Bus Station
3,Amrut Nagar,Café,Indian Restaurant,Lounge,Clothing Store,Diner,Shopping Mall,Electronics Store,Pizza Place,Asian Restaurant,Fast Food Restaurant
4,Asalfa,Café,Grocery Store,Light Rail Station,Restaurant,Lounge,Food Truck,Factory,Multiplex,Convenience Store,Gym
...,...,...,...,...,...,...,...,...,...,...,...
91,Uttaran,Restaurant,Pizza Place,Indian Restaurant,Café,Food Court,Chinese Restaurant,Fast Food Restaurant,South American Restaurant,Bus Station,Sports Club
92,Vakola,Bakery,Smoke Shop,Indian Restaurant,Seafood Restaurant,Outdoors & Recreation,Restaurant,Lounge,Café,Chinese Restaurant,Asian Restaurant
93,Versova,Café,Bar,Pub,Indian Restaurant,Lounge,Pizza Place,Chinese Restaurant,Ice Cream Shop,Seafood Restaurant,Coffee Shop
94,Vikhroli Park Site,Indian Restaurant,Café,Lounge,Clothing Store,Restaurant,Pizza Place,Electronics Store,Fast Food Restaurant,Diner,Mediterranean Restaurant


## Lets Do clustering For neighbourhoods and venue * 5 Cluster*

In [0]:
from sklearn.cluster import KMeans

kclusters = 5

Mumbai_grouped_clustering = Mumbai_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Mumbai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 1, 3, 3, 4, 4, 3, 1, 1], dtype=int32)

In [0]:
neighborhoods_venues_sorted['Neighborhood']

0      Aarey Milk Colony
1            Akurli road
2                 Amboli
3            Amrut Nagar
4                 Asalfa
             ...        
91               Uttaran
92                Vakola
93               Versova
94    Vikhroli Park Site
95         samarth NAgar
Name: Neighborhood, Length: 96, dtype: object

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,latitude,longitude
0,Aarey Milk Colony,Monument / Landmark,Fast Food Restaurant,Indian Restaurant,Lake,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,19.127931,72.847735
1,Akurli road,Ice Cream Shop,Indian Restaurant,Coffee Shop,Bakery,Food Truck,Sandwich Place,General Entertainment,Dessert Shop,Arts & Crafts Store,Film Studio,19.115287,72.861808
2,Amboli,Bakery,Sandwich Place,Indian Restaurant,Camera Store,Chinese Restaurant,Falafel Restaurant,Athletics & Sports,Asian Restaurant,Coffee Shop,Bus Station,19.128292,72.830193
3,Amrut Nagar,Café,Indian Restaurant,Lounge,Clothing Store,Diner,Shopping Mall,Electronics Store,Pizza Place,Asian Restaurant,Fast Food Restaurant,19.128794,72.825554
4,Asalfa,Café,Grocery Store,Light Rail Station,Restaurant,Lounge,Food Truck,Factory,Multiplex,Convenience Store,Gym,19.111100,72.865600


In [0]:
Mumbai_merged.shape

(99, 13)

In [0]:
Mumbai_merged = neighborhoods_venues_sorted

# add clustering labels
Mumbai_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

Mumbai_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Aarey Milk Colony,Monument / Landmark,Fast Food Restaurant,Indian Restaurant,Lake,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,4
1,Akurli road,Ice Cream Shop,Indian Restaurant,Coffee Shop,Bakery,Food Truck,Sandwich Place,General Entertainment,Dessert Shop,Arts & Crafts Store,Film Studio,4
2,Amboli,Bakery,Sandwich Place,Indian Restaurant,Camera Store,Chinese Restaurant,Falafel Restaurant,Athletics & Sports,Asian Restaurant,Coffee Shop,Bus Station,1
3,Amrut Nagar,Café,Indian Restaurant,Lounge,Clothing Store,Diner,Shopping Mall,Electronics Store,Pizza Place,Asian Restaurant,Fast Food Restaurant,3
4,Asalfa,Café,Grocery Store,Light Rail Station,Restaurant,Lounge,Food Truck,Factory,Multiplex,Convenience Store,Gym,3


In [0]:
#Appending Lag Lat dataframe to above dataset

## Appending Lag Lat dataframe to above dataset

In [0]:
lag_lat=pd.read_csv("//content/mumbai_csv.csv")
dflag=lag_lat['latitude']
dflat=lag_lat['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
Mumbai_sorted=pd.concat([neighborhoods_venues_sorted, lag_lat], axis=1)
Mumbai_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,latitude,longitude
0,Aarey Milk Colony,Monument / Landmark,Fast Food Restaurant,Indian Restaurant,Lake,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,4.0,19.127931,72.847735
1,Akurli road,Ice Cream Shop,Indian Restaurant,Coffee Shop,Bakery,Food Truck,Sandwich Place,General Entertainment,Dessert Shop,Arts & Crafts Store,Film Studio,4.0,19.115287,72.861808
2,Amboli,Bakery,Sandwich Place,Indian Restaurant,Camera Store,Chinese Restaurant,Falafel Restaurant,Athletics & Sports,Asian Restaurant,Coffee Shop,Bus Station,1.0,19.128292,72.830193
3,Amrut Nagar,Café,Indian Restaurant,Lounge,Clothing Store,Diner,Shopping Mall,Electronics Store,Pizza Place,Asian Restaurant,Fast Food Restaurant,3.0,19.128794,72.825554
4,Asalfa,Café,Grocery Store,Light Rail Station,Restaurant,Lounge,Food Truck,Factory,Multiplex,Convenience Store,Gym,3.0,19.111100,72.865600


In [0]:
Mumbai_sorted.shape

(99, 14)

# DATA Analysis  start

### Creating Folium Map of Cluster we just define.

In [0]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = Mumbai_sorted.loc[0, 'latitude']
longitude = Mumbai_sorted.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, poi,cluster in zip(Mumbai_sorted['latitude'],Mumbai_sorted['longitude'],Mumbai_sorted['Neighborhood'],Mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

![alt text](https://github.com/samir-cell/fantastic-datascience-Coursera_Capstone/blob/master/img/cluster%20venue.JPG?raw=true)

## Show Data of common venue  from each cluster

In [0]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 0, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
45,Indian Restaurant,English Restaurant,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,0
55,Indian Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,Fast Food Restaurant,Farmers Market,0


In [0]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 1, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Bakery,Chinese Restaurant,Falafel Restaurant,Athletics & Sports,Asian Restaurant,Coffee Shop,Bus Station,1
8,Coffee Shop,Cocktail Bar,Chinese Restaurant,Café,Fast Food Restaurant,Boat or Ferry,Food Truck,1
9,Coffee Shop,Clothing Store,Indian Restaurant,Pizza Place,Smoke Shop,Shopping Mall,Electronics Store,1
12,Convenience Store,Grocery Store,Hotel,Indian Chinese Restaurant,Indian Restaurant,Lake,Lounge,1
18,Coffee Shop,Sandwich Place,Arts & Crafts Store,Indian Restaurant,Soccer Field,Food Truck,Flea Market,1
21,Fast Food Restaurant,Seafood Restaurant,Shopping Mall,Coffee Shop,Bar,Vegetarian / Vegan Restaurant,Convenience Store,1
26,Train Station,Indian Restaurant,Pizza Place,Plaza,Dessert Shop,Multiplex,Fast Food Restaurant,1
27,Fast Food Restaurant,Coffee Shop,Donut Shop,Bar,Vegetarian / Vegan Restaurant,Pizza Place,Italian Restaurant,1
28,Indian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Pizza Place,Sandwich Place,Electronics Store,1
29,Indian Restaurant,Scenic Lookout,Coffee Shop,Sandwich Place,Lounge,Tea Room,Pub,1


In [0]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 2, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
65,Park,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Film Studio,2


In [0]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 3, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
3,Café,Diner,Shopping Mall,Electronics Store,Pizza Place,Asian Restaurant,Fast Food Restaurant,3
4,Café,Lounge,Food Truck,Factory,Multiplex,Convenience Store,Gym,3
7,Indian Restaurant,Bakery,Pizza Place,Bar,Snack Place,Deli / Bodega,Gourmet Shop,3
11,Indian Restaurant,Pizza Place,Diner,Café,Asian Restaurant,Multiplex,Restaurant,3
16,Bar,Asian Restaurant,Lounge,Pizza Place,Vegetarian / Vegan Restaurant,Electronics Store,Café,3
20,Indian Restaurant,Punjabi Restaurant,Bakery,Chinese Restaurant,Asian Restaurant,Sandwich Place,Restaurant,3
23,Pub,Lounge,Electronics Store,Seafood Restaurant,Chinese Restaurant,Coffee Shop,Juice Bar,3
24,Indian Restaurant,Food Court,Bar,Lake,Chinese Restaurant,General Entertainment,Multiplex,3
33,Indian Restaurant,Pizza Place,Ice Cream Shop,Bakery,Restaurant,Café,Pub,3
34,Asian Restaurant,Building,Café,Concert Hall,Food Court,Film Studio,Fish & Chips Shop,3


In [0]:
Mumbai_merged.loc[Mumbai_merged['Cluster Labels'] == 4, Mumbai_merged.columns[[1] + list(range(5, Mumbai_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Monument / Landmark,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Film Studio,4
1,Ice Cream Shop,Food Truck,Sandwich Place,General Entertainment,Dessert Shop,Arts & Crafts Store,Film Studio,4
5,Indian Restaurant,Hotel,Dessert Shop,Clothing Store,Chinese Restaurant,Plaza,Fast Food Restaurant,4
6,Indian Restaurant,Fast Food Restaurant,Diner,Lounge,Café,Chinese Restaurant,Dessert Shop,4
10,Indian Restaurant,Multiplex,Punjabi Restaurant,Gym / Fitness Center,Gym,Bus Station,Event Space,4
13,Indian Restaurant,Plaza,Café,Motorcycle Shop,Gym / Fitness Center,Pizza Place,Farmers Market,4
14,Indian Restaurant,Bakery,Ice Cream Shop,Fast Food Restaurant,Restaurant,Juice Bar,Food Truck,4
15,Indian Restaurant,Juice Bar,Chinese Restaurant,Bakery,Middle Eastern Restaurant,Fast Food Restaurant,Restaurant,4
17,Indian Restaurant,Zoo,Asian Restaurant,Nightclub,Pizza Place,Platform,Chinese Restaurant,4
19,Indian Restaurant,Bakery,Asian Restaurant,Bar,Garden,Seafood Restaurant,Punjabi Restaurant,4


## Analysis result:
1.Indian Restaurant are present in each cluster (1st common venue ) 
2.coffe shop is popular among Mumbai people. (1st common venue in some neighborhood)
 we can locate coffee shop location for opening new coffee shop.



# Lets Finding Best location or Neighborhood for new coffee shop.

In [0]:
Mumbai_grouped = Mumbai_onehot.groupby('Neighborhood').mean().reset_index()
Mumbai_grouped

,Neighborhood,Zoo,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Aarey Milk Colony,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,Akurli road,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.133333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
2,Amboli,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.055556,0.055556,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.055556,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
3,Amrut Nagar,0.0,0.018868,0.0,0.0,0.0,0.0,0.0,0.018868,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.037736,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.018868,0.0,0.018868,0.000000,0.000000,0.018868,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.037736,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.018868,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.018868,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.037736,0.0,0.0,0.000000,0.018868,0.0
4,Asalfa,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Uttaran,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000

In [0]:
len(Mumbai_grouped[Mumbai_grouped["Coffee Shop"] > 0])
mumbai_rest = Mumbai_grouped[["Neighborhood","Coffee Shop"]]
mumbai_rest

,Neighborhood,Coffee Shop
0,Aarey Milk Colony,0.000000
1,Akurli road,0.133333
2,Amboli,0.055556
3,Amrut Nagar,0.018868
4,Asalfa,0.000000
...,...,...
91,Uttaran,0.000000
92,Vakola,0.000000
93,Versova,0.026316
94,Vikhroli Park Site,0.013889


## Clustering venue 

In [0]:
kclusters = 5

mumbai_clustering = mumbai_rest.drop(["Neighborhood"], 1)
mumbai_clustering.head()

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, n_init=12).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 3, 1, 0, 0, 0, 2, 4], dtype=int32)

### Getting Mean for each  Neighborhoods of coffee shop.

In [0]:
merged = mumbai_rest.copy()

# add clustering labels
merged["Category"] = kmeans.labels_
#merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
merged.head()

,Neighborhood,Coffee Shop,Category
0,Aarey Milk Colony,0.000000,1
1,Akurli road,0.133333,2
2,Amboli,0.055556,0
3,Amrut Nagar,0.018868,3
4,Asalfa,0.000000,1


### Create Folium Map to analys cluster 

In [0]:
lag_lat=pd.read_csv("//content/mumbai_csv.csv")
dflag=lag_lat['latitude']
dflat=lag_lat['longitude']
lag_lat= pd.concat([dflag, dflat], axis=1)
#lag_lat.head()
Mumbai_w_Loc=pd.concat([merged, lag_lat], axis=1)
Mumbai_w_Loc.head()

,Neighborhood,Coffee Shop,Category,latitude,longitude
0,Aarey Milk Colony,0.000000,1.0,19.127931,72.847735
1,Akurli road,0.133333,2.0,19.115287,72.861808
2,Amboli,0.055556,0.0,19.128292,72.830193
3,Amrut Nagar,0.018868,3.0,19.128794,72.825554
4,Asalfa,0.000000,1.0,19.111100,72.865600


In [0]:
#map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
import matplotlib.cm as cm
from matplotlib import colors
latitude = Mumbai_w_Loc.loc[0, 'latitude']
longitude = Mumbai_w_Loc.loc[0, 'longitude']
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to map
for lat, lng, poi,cluster in zip(Mumbai_w_Loc['latitude'],Mumbai_w_Loc['longitude'],Mumbai_w_Loc['Neighborhood'],merged['Category']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster -1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
    
map_clusters

![alt text](https://github.com/samir-cell/fantastic-datascience-Coursera_Capstone/blob/master/img/cluster%20final.JPG?raw=true)

In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 0]
#Sort
Mumbai_w_Loc.sort_values(["Category"], inplace=True, ascending=False)
Mumbai_w_Loc

,Neighborhood,Coffee Shop,Category,latitude,longitude
86,Sunder Nagar,0.096386,4.0,19.027200,72.867100
58,Mira Road,0.080460,4.0,19.147500,72.929900
17,Dagdi Chawl,0.076923,4.0,19.067115,72.865724
72,Pant Nagar,0.086957,4.0,19.001048,72.819427
9,Bangur Nagar,0.104167,4.0,19.136918,72.869128
...,...,...,...,...,...
35,Irla,0.044444,0.0,19.213220,72.846602
61,NESCO colony,0.062500,0.0,19.137486,72.813990
96,NaN,NaN,NaN,18.959352,72.829914
97,NaN,NaN,NaN,18.991424,72.836459


Category 1: Neighborhoods with very low number of Coffee Shop

In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 0.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
87,Thakkar Bappa Colony,0.052632,0.0,18.978000,72.828300
31,Hindu colony,0.066667,0.0,19.150300,72.853000
28,Gowalia Tank,0.047059,0.0,19.173853,72.867094
42,Jogeshwari West,0.049383,0.0,19.198200,72.873700
16,D.N. Nagar,0.058140,0.0,19.319455,72.897241
46,Khar Danda,0.053571,0.0,19.062785,72.829391
5,Ballard Estate,0.060606,0.0,19.143189,72.824081
53,Maharastra Nagar,0.052632,0.0,19.079273,72.861267
7,Bandra Reclamation,0.052083,0.0,19.184102,72.943766
6,Bandra Kurla Complex,0.059701,0.0,19.117220,72.882342



Category 2: Neighborhoods with low number of coffee shop

In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 1.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
32,Hindustan Naka,0.000000,1.0,19.136394,72.837382
11,Chakala,0.000000,1.0,19.108221,72.883582
12,Chandivali,0.000000,1.0,19.129762,72.821378
15,Chor Bazaar,0.000000,1.0,19.056300,72.852200
19,Dayanand Saraswati Marg,0.000000,1.0,19.058040,72.847258
20,Deonar,0.000000,1.0,19.061218,72.844873
24,Gandhi Nagar,0.000000,1.0,19.051060,72.833053
60,Motilal Nagar,0.000000,1.0,19.140029,72.924671
43,Kalina,0.000000,1.0,19.200273,72.876951
95,samarth NAgar,0.000000,1.0,18.951594,72.825696


In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 2.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
63,Naigaon,0.178571,2.0,19.223281,72.854437
62,Nahur,0.142857,2.0,19.143321,72.925191
78,Rani Sati Marg,0.153846,2.0,19.116838,72.940120
34,Indian Institute of Technology Bombay campus,0.125000,2.0,19.214119,72.825865
79,Sahar,0.131148,2.0,19.104371,72.914010
18,Damu nagar,0.166667,2.0,19.042718,72.819132
1,Akurli road,0.133333,2.0,19.115287,72.861808
50,Land's End,0.128205,2.0,19.179515,72.862397
51,Lokandwala township,0.142857,2.0,19.183766,72.849538
57,Marol,0.142857,2.0,19.120547,72.820843


In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 3.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
21,Dindoshi,0.041667,3.0,19.045294,72.820183
27,Gokuldham,0.042553,3.0,19.168814,72.833678
30,Hanuman nagar,0.036364,3.0,19.170900,19.170900
89,Thakur village,0.031250,3.0,18.909766,72.809831
22,Fanas Wadi,0.035088,3.0,19.064913,72.827033
3,Amrut Nagar,0.018868,3.0,19.128794,72.825554
90,Utkarsh Nagar,0.023256,3.0,19.019962,72.847893
93,Versova,0.026316,3.0,18.955600,72.820300
81,Saki Naka,0.028571,3.0,19.053044,72.900896
67,Old Town,0.040000,3.0,19.100845,72.911820


In [0]:
Mumbai_w_Loc.loc[Mumbai_w_Loc['Category'] == 4.0]

,Neighborhood,Coffee Shop,Category,latitude,longitude
86,Sunder Nagar,0.096386,4.0,19.027200,72.867100
58,Mira Road,0.080460,4.0,19.147500,72.929900
17,Dagdi Chawl,0.076923,4.0,19.067115,72.865724
72,Pant Nagar,0.086957,4.0,19.001048,72.819427
9,Bangur Nagar,0.104167,4.0,19.136918,72.869128
13,Charkop,0.100000,4.0,19.130252,72.821377


## Conclusion :

Based on the both Clusters formed it would be a good idea to open a new coffee shop in Clusters 0 or 1 & 2 since the other clusters already have coffee shop and Restaurants in their vicinities. cluster 0 and "NAN" value having no competition has possibility is there people are not interested at coffee shop. 




```
Note : result is besed on only venue data AND PRESENT  COFFEE SHOP DATA . we can predit more accurate data using population and rating.
```

